In [1]:
# Transformers installation
#! pip install transformers
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git
#need to downgrade protobuf to a compatible version (e.g., 3.20.x). if running with cpu instead of gpu environment
#!pip install protobuf==6.32.*
#!pip install torch
#By default, Hugging Face pipeline tries PyTorch first, and only if PyTorch isn’t available does 
#it fall back to TensorFlow
#!pip install tf-keras 
#This installs the backwards-compatible version of Keras that works with TensorFlow and Hugging Face.
#install Jupyter widgets support:
#!pip install ipywidgets
#the model repo supports XetHub storage, which is faster.
#!pip install huggingface_hub[hf_xet]
#Install GPU version of TensorFlow
#!pip install tensorflow[and-cuda]
#Verify GPU is detected:
#import tensorflow as tf
#print(tf.config.list_physical_devices("GPU"))
#import sys
#print(sys.executable)

In [37]:
import torch as pt
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

PyTorch version: 2.2.0+cu121
CUDA version: 12.1
GPU available: True
GPU name: NVIDIA GeForce RTX 3050 A Laptop GPU


In [2]:
import transformers
print(transformers.__version__)

4.56.1


In [5]:
from transformers import pipeline #using pipeline 2 things get loaded
#pipeline helps me to call a pre trained model like bert,gpt2 etc
#in this classifier pretrained model will get downloaded on sentiment-analysis
#distilbert-base model used finetuned on sst-2-english -(dataset used) , uncased means lowercase dataset used
# Make sure GPU is available
device = 0 if torch.cuda.is_available() else -1

# Load a small model (sentiment analysis)
classifier1 = pipeline("sentiment-analysis", device=device,model="distilbert/distilbert-base-uncased-finetuned-sst-2-english") 
result = classifier1("I finally fixed my PyTorch and Transformers setup! 🎉")
print(result)

Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9774676561355591}]


In [6]:
classifier1('We are very happy to show you the 🤗 Transformers library.')

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [7]:
classifier1('The pizza is not that great but the crust is awesome.')

[{'label': 'POSITIVE', 'score': 0.9998461008071899}]

In [8]:
results = classifier1(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


In [9]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

Device set to use cuda:0


In [10]:
classifier("Esperamos que no lo odie.")

[{'label': '3 stars', 'score': 0.33688199520111084}]

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#whenever any kind of model we have to follow  below  mentioned process
#AutoTokenizer- take test data and convert into numerical-embeddings like word2vec
#after embeddings generation ,then only model will classify 
#TFAutoModelForSequenceClassification-if we are using tensor flow it will be used to download model itself.

In [16]:
#AutoModelForSequenceClassification for pytorch
#Use from_pt=True only if you want to convert a PyTorch model to TensorFlow.If you’re running in PyTorch only, just omit the flag.
#device=0 ensures the pipeline runs on GPU.
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
# This model only exists in PyTorch, so we use the `from_pt` flag to import that model in TensorFlow.
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Use pipeline on GPU if available
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer,device=0)

Device set to use cuda:0


In [17]:
classifier("I am a bad boy")

[{'label': '1 star', 'score': 0.7558020353317261}]

In [18]:
classifier("I am a good boy")

[{'label': '4 stars', 'score': 0.42292681336402893}]

In [ ]:
#Inference on a batch (no pipeline)
#understand exactly what happens: tokenize → move to device(gpu) → forward pass → softmax → labels.

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [21]:
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")

In [22]:
#every word is given a numerical number-indexes works as id to words
print(inputs)

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [24]:
#padding to make all sentences of equal length of 512
#pt_batch is a BatchEncoding (basically a dict with input_ids, attention_mask, etc.).
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt" #"pt"-- use "pt" for PyTorch tensors , "tf"-tensorflow tensors
)

In [28]:
# Show tokenized tensors
for key, value in pt_batch.items():
    print(f"{key}: {value.numpy().tolist()}")

input_ids: [[101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


In [41]:
# labels must be torch.LongTensor (class indices 0..4 for this model)
labels = torch.tensor([1, 0], dtype=torch.long)
#torch.tensor([1, 0], dtype=torch.long) → creates a tensor of class indices.
#(e.g., 1 = 2 stars, 0 = 1 star for this model).

In [42]:
#In PyTorch, Hugging Face models don’t accept a whole BatchEncoding object directly.
#we need to unpack the dict into keyword arguments.
# not this pt_outputs = pt_model(pt_batch)
# Forward pass (unpack with **)
# unpack the dict and pass labels
pt_outputs = pt_model(**pt_batch, labels=labels)
#The model expects arguments like model(input_ids=..., attention_mask=...).
#Using **pt_batch unpacks the dict into those arguments.

In [43]:
#pt_model(**pt_batch, labels=labels) → automatically computes cross-entropy loss + returns logits.
#pt_outputs.loss → scalar loss (use in training).
#pt_outputs.logits → raw predictions per class
print(pt_outputs)

SequenceClassifierOutput(loss=tensor(0.3167, grad_fn=<NllLossBackward0>), logits=tensor([[-4.0833,  4.3364],
        [ 0.0818, -0.0418]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [44]:
print("Loss:", pt_outputs.loss.item())
print("Logits:", pt_outputs.logits)

Loss: 0.3167364001274109
Logits: tensor([[-4.0833,  4.3364],
        [ 0.0818, -0.0418]], grad_fn=<AddmmBackward0>)


In [45]:
#import tensorflow as tf #for tensorflow
#tf_predictions = tf.nn.softmax(pt_outputs[0], axis=-1)
import torch.nn.functional as ActivationFunc
# Apply softmax on logits to get probabilities
pt_predictions = ActivationFunc.softmax(pt_outputs.logits, dim=-1)

In [46]:
print(pt_predictions)

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward0>)


In [49]:
save_directory = "./saved_model"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [52]:
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)

In [61]:
pt_outputs = pt_model(**pt_batch, output_hidden_states=True)
all_hidden_states, all_attentions = pt_outputs[-2:]

In [58]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [60]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=10)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
